<a id='home'></a>
### purpose

train lfmm and RDA on the multivariate sim with nuisance envs

### notes

lfmm code based on 02_analysis/05_multivariate_sims/04_train_lfmm.ipynb

rda code based on 02_analysis/05_multivariate_sims/06_train_RDA.ipynb

### outline

1. [lfmm](#lfmm)

1. [RDA](#rda)

In [1]:
from pythonimports import *

import MVP_summary_functions as mvp

lview, dview = get_client(cluster_id='1694097663-r97i', profile='lotterhos')

outerdir = '/home/b.lind/offsets/run_20220919_nuisance'
tdir = f'{outerdir}/tutorial'

mvp.latest_commit()
session_info.show()

36 36
#########################################################
Today:	September 07, 2023 - 10:47:35 EDT
python version: 3.8.5
conda env: mvp_env

Current commit of pythonimports:
commit 9493334506a1f2926fba0734fa74ba2f87194611  
Author: Brandon Lind <lind.brandon.m@gmail.com>  
Date:   Fri Jun 30 09:41:00 2023 -0400

Current commit of MVP_offsets:
commit 34fec27ce185eb670eb16124a5255c9d4964ee70  
Author: Brandon Lind <lind.brandon.m@gmail.com>  
Date:   Tue Sep 5 11:47:40 2023 -0400
#########################################################



<a id='lfmm'></a>
# lfmm

[top](#home)

In [2]:
mvp_dir = '/home/b.lind/code/MVP-offsets/01_src'
# path to lfmm offset training script
training_script = f'{mvp_dir}/MVP_complex_sims_process_lfmm.R'

op.exists(training_script)

True

In [3]:
indir = makedir(op.join(tdir, 'lfmm2/lfmm_infiles'))
outdir = makedir(op.join(tdir, f'lfmm2/lfmm_outfiles'))
shdir = makedir(op.join(tdir, 'lfmm2/lfmm_shfiles'))

shdir

'/home/b.lind/offsets/run_20220919_nuisance/tutorial/lfmm2/lfmm_shfiles'

In [4]:
run = op.basename(outerdir)

run

'run_20220919_nuisance'

## main

### mvp10.create_current_envfiles

In [5]:
# get nuis data
nuis_data = pd.read_table('/home/b.lind/offsets/run_20220919_tutorial/tutorial/My_Individuals_nuis_vals.txt')

print(nuis_data.shape)

nuis_data.head()

(1000, 3)


,ISO,TSsd,PSsd
0,-0.120444,0.769052,-0.552302
1,-0.704655,-0.925024,-1.011409
2,-0.704655,-0.782171,-0.781856
3,1.632190,-1.447701,-0.475785
4,0.463767,-2.101467,0.212875


In [6]:
# get adaptive envs
all_data = pd.read_table('/home/b.lind/offsets/run_20220919_tutorial/tutorial/My_Individuals.txt')

causal_data = all_data[[col for col in all_data.columns if col.startswith('env')]].copy()

causal_data

,env1_mat,env2_MTWetQ,env3_MTDQ,env4_PDM,env5_PwarmQ,env6_PWM
0,0.762432,-0.162238,0.261281,-0.332078,-0.471502,-0.477821
1,-0.339330,-0.407667,-0.058296,0.288030,0.198810,0.017680
2,-0.567733,0.090928,-0.414353,0.234899,0.230255,-0.053511
3,-0.245860,-0.200759,-0.267455,-0.265260,-0.389828,-0.458979
4,-0.121500,-0.306379,0.702858,0.211275,-0.352307,0.212751
...,...,...,...,...,...,...
995,0.476928,-0.216205,0.401987,-0.377651,-0.607663,-0.374126
996,-0.320404,-0.411335,-0.038372,0.238443,0.145508,-0.007439
997,0.225111,0.401138,-0.016827,-0.088830,-0.017708,-0.215783
998,-0.028901,0.372221,-0.533756,-0.467916,0.559185,0.267799


In [7]:
# combine nuis data and causal envs
envdata = pd.merge(nuis_data, causal_data, left_index=True, right_index=True)

print(envdata.shape)

envdata.head()

(1000, 9)


,ISO,TSsd,PSsd,env1_mat,env2_MTWetQ,env3_MTDQ,env4_PDM,env5_PwarmQ,env6_PWM
0,-0.120444,0.769052,-0.552302,0.762432,-0.162238,0.261281,-0.332078,-0.471502,-0.477821
1,-0.704655,-0.925024,-1.011409,-0.339330,-0.407667,-0.058296,0.288030,0.198810,0.017680
2,-0.704655,-0.782171,-0.781856,-0.567733,0.090928,-0.414353,0.234899,0.230255,-0.053511
3,1.632190,-1.447701,-0.475785,-0.245860,-0.200759,-0.267455,-0.265260,-0.389828,-0.458979
4,0.463767,-2.101467,0.212875,-0.121500,-0.306379,0.702858,0.211275,-0.352307,0.212751


In [8]:
# save
lfmm_envfile = op.join(indir, 'tutorial_lfmm_env_ntraits-6.txt')

envdata.to_csv(lfmm_envfile, sep='\t', index=False, header=False)

lfmm_envfile

'/home/b.lind/offsets/run_20220919_nuisance/tutorial/lfmm2/lfmm_infiles/tutorial_lfmm_env_ntraits-6.txt'

### mvp10.create_poplabels

In [9]:
og_poplabel = '/home/b.lind/offsets/run_20220919_tutorial/lfmm2/lfmm_infiles/tutorial_poplabels.txt'

poplabel_file = f'{indir}/tutorial_poplabels.txt'

if not op.exists(poplabel_file):
    os.symlink(og_poplabel, poplabel_file)

In [10]:
op.exists(poplabel_file)

True

In [11]:
!wc -l $poplabel_file

1000 /home/b.lind/offsets/run_20220919_nuisance/tutorial/lfmm2/lfmm_infiles/tutorial_poplabels.txt


### mvp10.create_transplant_envfiles

In [12]:
envdata.shape

(1000, 9)

In [13]:
all_data

,ind_index,x,y,phenotype1_mat,phenotype2_MTWetQ,phenotype3_MTDQ,phenotype4_PDM,phenotype5_PwarmQ,phenotype6_PWM,env1_mat,env2_MTWetQ,env3_MTDQ,env4_PDM,env5_PwarmQ,env6_PWM,subpopID,lon,lat
0,33,0.406184,0.233272,0.737967,-0.191421,0.142992,-0.131894,-0.452497,-0.541651,0.762432,-0.162238,0.261281,-0.332078,-0.471502,-0.477821,25,5.0,3.0
1,34,0.425697,0.837158,-0.274081,-0.428332,-0.012880,0.221058,0.229296,0.066597,-0.339330,-0.407667,-0.058296,0.288030,0.198810,0.017680,85,5.0,9.0
2,44,0.671673,0.581744,-0.676326,0.086629,-0.470671,0.218813,0.260098,-0.103465,-0.567733,0.090928,-0.414353,0.234899,0.230255,-0.053511,57,7.0,6.0
3,45,0.017466,0.329922,-0.283221,-0.403942,-0.333718,-0.200554,-0.345915,-0.410001,-0.245860,-0.200759,-0.267455,-0.265260,-0.389828,-0.458979,31,1.0,4.0
4,46,0.069744,0.121221,0.057616,-0.303201,0.731600,0.195679,-0.367110,0.196296,-0.121500,-0.306379,0.702858,0.211275,-0.352307,0.212751,11,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,8575,0.117890,0.278449,-0.070241,-0.367056,-0.061876,-0.308041,-0.468111,-0.508108,0.476928,-0.216205,0.401987,-0.377651,-0.607663,-0.374126,22,2.0,3.0
996,8580,0.464067,0.792001,-0.050435,-0.342693,0.128758,0.107181,-0.061796,-0.081682,-0.320404,-0.411335,-0.038372,0.238443,0.145508,-0.007439,75,5.0,8.0
997,8582,0.158645,0.682325,0.354088,0.493751,0.064744,-0.243331,-0.164916,-0.244379,0.225111,0.401138,-0.016827,-0.088830,-0.017708,-0.215783,62,2.0,7.0
998,8585,0.894104,0.757964,-0.009027,0.424444,-0.575177,-0.514613,0.586778,0.266235,-0.028901,0.372221,-0.533756,-0.467916,0.559185,0.267799,79,9.0,8.0


In [14]:
all_data[[col for col in all_data.columns if col in envdata.columns]]

,env1_mat,env2_MTWetQ,env3_MTDQ,env4_PDM,env5_PwarmQ,env6_PWM
0,0.762432,-0.162238,0.261281,-0.332078,-0.471502,-0.477821
1,-0.339330,-0.407667,-0.058296,0.288030,0.198810,0.017680
2,-0.567733,0.090928,-0.414353,0.234899,0.230255,-0.053511
3,-0.245860,-0.200759,-0.267455,-0.265260,-0.389828,-0.458979
4,-0.121500,-0.306379,0.702858,0.211275,-0.352307,0.212751
...,...,...,...,...,...,...
995,0.476928,-0.216205,0.401987,-0.377651,-0.607663,-0.374126
996,-0.320404,-0.411335,-0.038372,0.238443,0.145508,-0.007439
997,0.225111,0.401138,-0.016827,-0.088830,-0.017708,-0.215783
998,-0.028901,0.372221,-0.533756,-0.467916,0.559185,0.267799


In [15]:
pop_envdata = pd.merge(all_data, nuis_data,
                       left_index=True,
                       right_index=True).groupby('subpopID')[envdata.columns].apply(np.mean)

pop_envdata

,ISO,TSsd,PSsd,env1_mat,env2_MTWetQ,env3_MTDQ,env4_PDM,env5_PwarmQ,env6_PWM
subpopID,,,,,,,,,
1,0.009381,-2.245441,1.003559,0.186225,-0.226935,0.861531,0.221974,-0.363974,0.761569
2,0.239071,-1.967663,0.518946,0.070353,-0.265649,0.799727,0.169512,-0.417965,0.315295
3,0.580610,-0.963343,-0.246231,0.264633,-0.248372,0.375248,-0.172690,-0.586408,-0.320803
4,0.639031,-0.164371,-0.399267,0.371616,0.167485,0.063118,-0.465978,-0.656053,-0.625240
5,-0.036985,-0.291426,-0.661614,0.244613,0.117437,-0.010939,-0.291486,-0.528748,-0.537376
...,...,...,...,...,...,...,...,...,...
96,0.113241,-0.239327,-0.054937,-0.288220,0.261489,-0.317123,-0.018548,0.228030,-0.045559
97,0.881061,0.598108,1.480884,-0.116570,0.323553,-0.231351,-0.269762,0.344912,0.059459
98,1.587251,1.669095,2.461321,0.093915,0.400397,-0.150975,-0.408936,0.588880,0.274592


In [16]:
lfmm_envfile

'/home/b.lind/offsets/run_20220919_nuisance/tutorial/lfmm2/lfmm_infiles/tutorial_lfmm_env_ntraits-6.txt'

In [17]:
envdata.shape

(1000, 9)

In [18]:
# mvp10.create_transplant_envfiles

garden_files = []

for subpop_id, env_vals in pop_envdata.iterrows():
    X_new = envdata.copy()
    
    # overwrite env vals with single value of transplant env
    for col in X_new:
        X_new[col] = env_vals[col]
        
    lfmm_envfile_new = lfmm_envfile.replace('.txt',
                                           f'_{str(subpop_id).zfill(3)}.txt')
    
    X_new.to_csv(lfmm_envfile_new, sep='\t', index=False, header=False)
    
    garden_files.append(lfmm_envfile_new)    
    
len(garden_files)

100

In [19]:
X_new

,ISO,TSsd,PSsd,env1_mat,env2_MTWetQ,env3_MTDQ,env4_PDM,env5_PwarmQ,env6_PWM
0,1.137857,3.232988,2.967515,0.055334,0.422488,-0.553991,-0.472096,0.604139,0.32116
1,1.137857,3.232988,2.967515,0.055334,0.422488,-0.553991,-0.472096,0.604139,0.32116
2,1.137857,3.232988,2.967515,0.055334,0.422488,-0.553991,-0.472096,0.604139,0.32116
3,1.137857,3.232988,2.967515,0.055334,0.422488,-0.553991,-0.472096,0.604139,0.32116
4,1.137857,3.232988,2.967515,0.055334,0.422488,-0.553991,-0.472096,0.604139,0.32116
...,...,...,...,...,...,...,...,...,...
995,1.137857,3.232988,2.967515,0.055334,0.422488,-0.553991,-0.472096,0.604139,0.32116
996,1.137857,3.232988,2.967515,0.055334,0.422488,-0.553991,-0.472096,0.604139,0.32116
997,1.137857,3.232988,2.967515,0.055334,0.422488,-0.553991,-0.472096,0.604139,0.32116
998,1.137857,3.232988,2.967515,0.055334,0.422488,-0.553991,-0.472096,0.604139,0.32116


### create_adaptive_and_neutral_files

these are the same as the original tutorial run

In [20]:
loci_files = fs('/home/b.lind/offsets/run_20220919_tutorial/lfmm2/lfmm_infiles', pattern='loci')

loci_files

['/home/b.lind/offsets/run_20220919_tutorial/lfmm2/lfmm_infiles/tutorial_adaptive_loci.txt',
 '/home/b.lind/offsets/run_20220919_tutorial/lfmm2/lfmm_infiles/tutorial_all_loci.txt',
 '/home/b.lind/offsets/run_20220919_tutorial/lfmm2/lfmm_infiles/tutorial_neutral_loci.txt']

In [21]:
dst_files = {}
for src in loci_files:
    basename = op.basename(src)
    dst = f'{indir}/{basename}'
    
    marker_set = basename.split("_")[1]
    
    dst_files[marker_set] = dst
    
    if not op.exists(dst):
        os.symlink(src, dst)
        
dst_files

{'adaptive': '/home/b.lind/offsets/run_20220919_nuisance/tutorial/lfmm2/lfmm_infiles/tutorial_adaptive_loci.txt',
 'all': '/home/b.lind/offsets/run_20220919_nuisance/tutorial/lfmm2/lfmm_infiles/tutorial_all_loci.txt',
 'neutral': '/home/b.lind/offsets/run_20220919_nuisance/tutorial/lfmm2/lfmm_infiles/tutorial_neutral_loci.txt'}

### mvp10.create_shfiles

In [22]:
# mvp10.create_shfiles(lfmm_envfiles, poplabel_file, garden_files, locus_files, thresh=36)

adaptive_file = dst_files['adaptive']
neutral_file = dst_files['neutral']

# how to know when I'm done creating commands = num_gardens * num_marker_sets
expected_jobcount = len(garden_files) * (len(loci_files))  # hard-coded 1 as "all" loci below

expected_jobcount

300

In [23]:
training_script

'/home/b.lind/code/MVP-offsets/01_src/MVP_complex_sims_process_lfmm.R'

In [24]:
tdir

'/home/b.lind/offsets/run_20220919_nuisance/tutorial'

In [25]:
# creating a symlink from srcfile to geno_dst didn't work - trying now by saving with sep=' '
    # comment is probably referencing when jobs failed due to mem, I bet symlink would work fine
srcfile = '/home/b.lind/offsets/run_20220919_tutorial/tutorial/Genotypes.txt'

geno_dst = op.join(indir, 'Genotypes.lfmm')

df = pd.read_table(srcfile, delim_whitespace=True, header=None)

df.to_csv(geno_dst, sep=' ', index=False, header=False)

In [27]:
dst

'/home/b.lind/offsets/run_20220919_nuisance/tutorial/lfmm2/lfmm_infiles/Genotypes.lfmm'

In [28]:
thresh = 36

jobcount = 0  # to know when I'm done iterating
cmds = []  # list of training commands to parallelize for a given job
shfiles = []
for garden_file in pbar(garden_files):
    garden = op.basename(garden_file).split("_")[-1].split('.')[0]  # subpopID

    for cand_file, marker_set in zip(['None', adaptive_file, neutral_file], ['all', 'adaptive', 'neutral']):
        output_file = op.join(outdir, f'tutorial_lfmm_offsets_ntraits-9_{marker_set}_{garden}.txt')

        cmd = ' '.join(['Rscript',
                        training_script,
                        geno_dst,
                        lfmm_envfile,
                        garden_file,
                        poplabel_file,
                        output_file,
                        cand_file])
        cmds.append(cmd)
        jobcount += 1

        if len(cmds) == thresh or jobcount == expected_jobcount:
            batch_num = str(len(shfiles)).zfill(3)
            job = f'{run}_tutorial_lfmm_batch_{batch_num}'

            # determine amount of memory required (with cushion)
            if len(cmds) == thresh:
                mem = '175000M' 
            else:
                val = (len(cmds) * 4900) + 5000
                mem = f'{val}M'

            # write commands to a file that I can `cat` to GNU parallel
            cmd_file = op.join(indir, f'{job}.txt')
            with open(cmd_file, 'w') as o:
                o.write('\n'.join(cmds))

            # where to write sh text
            shfile = op.join(shdir, f'{job}.sh')

            # what text to write to sh
            text = f'''#!/bin/bash
#SBATCH --job-name={job}
#SBATCH --time=03:00:00
#SBATCH --mem={mem}
#SBATCH --partition=short
#SBATCH --nodes=1
#SBATCH --cpus-per-task=36
#SBATCH --output={job}_%j.out
#SBATCH --mail-user=b.lind@northeastern.edu
#SBATCH --mail-type=FAIL

source $HOME/.bashrc  # assumed that conda init is within .bashrc

module load parallel

conda activate MVP_env_R4.0.3

# train lfmm2 and calculate offset
cat {cmd_file} | parallel -j {len(cmds)} --progress --eta

conda activate mvp_env

cd {mvp_dir}

# re run any failed jobs from the cmd_file
python MVP_watch_for_failure_of_train_lfmm2_offset.py tutorial {shfile} {outerdir} {len(cmds)}

'''
            with open(shfile, 'w') as o:
                o.write(text)
            shfiles.append(shfile)

            cmds = []  # reset

len(shfiles)

100%|███████████████| 100/100 [00:00<00:00, 576.75it/s]


9

In [29]:
cmd.split()

['Rscript',
 '/home/b.lind/code/MVP-offsets/01_src/MVP_complex_sims_process_lfmm.R',
 '/home/b.lind/offsets/run_20220919_nuisance/tutorial/lfmm2/lfmm_infiles/Genotypes.lfmm',
 '/home/b.lind/offsets/run_20220919_nuisance/tutorial/lfmm2/lfmm_infiles/tutorial_lfmm_env_ntraits-6.txt',
 '/home/b.lind/offsets/run_20220919_nuisance/tutorial/lfmm2/lfmm_infiles/tutorial_lfmm_env_ntraits-6_100.txt',
 '/home/b.lind/offsets/run_20220919_nuisance/tutorial/lfmm2/lfmm_infiles/tutorial_poplabels.txt',
 '/home/b.lind/offsets/run_20220919_nuisance/tutorial/lfmm2/lfmm_outfiles/tutorial_lfmm_offsets_ntraits-6_neutral_100.txt',
 '/home/b.lind/offsets/run_20220919_nuisance/tutorial/lfmm2/lfmm_infiles/tutorial_neutral_loci.txt']

In [31]:
shfiles

['/home/b.lind/offsets/run_20220919_nuisance/tutorial/lfmm2/lfmm_shfiles/run_20220919_nuisance_tutorial_lfmm_batch_000.sh',
 '/home/b.lind/offsets/run_20220919_nuisance/tutorial/lfmm2/lfmm_shfiles/run_20220919_nuisance_tutorial_lfmm_batch_001.sh',
 '/home/b.lind/offsets/run_20220919_nuisance/tutorial/lfmm2/lfmm_shfiles/run_20220919_nuisance_tutorial_lfmm_batch_002.sh',
 '/home/b.lind/offsets/run_20220919_nuisance/tutorial/lfmm2/lfmm_shfiles/run_20220919_nuisance_tutorial_lfmm_batch_003.sh',
 '/home/b.lind/offsets/run_20220919_nuisance/tutorial/lfmm2/lfmm_shfiles/run_20220919_nuisance_tutorial_lfmm_batch_004.sh',
 '/home/b.lind/offsets/run_20220919_nuisance/tutorial/lfmm2/lfmm_shfiles/run_20220919_nuisance_tutorial_lfmm_batch_005.sh',
 '/home/b.lind/offsets/run_20220919_nuisance/tutorial/lfmm2/lfmm_shfiles/run_20220919_nuisance_tutorial_lfmm_batch_006.sh',
 '/home/b.lind/offsets/run_20220919_nuisance/tutorial/lfmm2/lfmm_shfiles/run_20220919_nuisance_tutorial_lfmm_batch_007.sh',
 '/home/

In [32]:
pids = sbatch(shfiles)

sbatching: 100%|███████████████| 9/9 [00:02<00:00,  4.37it/s]


In [33]:
create_watcherfile(pids, shdir, watcher_name='tutorial_nuisance_watcher')

['38489641']


'/home/b.lind/offsets/run_20220919_nuisance/tutorial/lfmm2/lfmm_shfiles/tutorial_nuisance_watcher.sh'

In [34]:
Squeue(p='short', states='PD').update(to_partition='long', num_jobs=0.5)

update: 100%|███████████████| 5/5 [00:00<00:00, 10.64it/s]


In [36]:
Squeue().watch()

🗒️  Queue Summary:

{'long': {'PD': 2, 'R': 3},
 'lotterhos': {'R': 2},
 'short': {'PD': 3, 'R': 11}}


sleeping:   0%|               | 0/5 [00:00<?, ?it/s]

KeyboardInterrupt after 0 seconds of sleep.


<a id='rda'></a>
# RDA

[top](#home)

In [1]:
from pythonimports import *

import MVP_summary_functions as mvp

mvp.latest_commit()
session_info.show()

#########################################################
Today:	September 07, 2023 - 11:25:09 EDT
python version: 3.8.5
conda env: mvp_env

Current commit of pythonimports:
commit 9493334506a1f2926fba0734fa74ba2f87194611  
Author: Brandon Lind <lind.brandon.m@gmail.com>  
Date:   Fri Jun 30 09:41:00 2023 -0400

Current commit of MVP_offsets:
commit 34fec27ce185eb670eb16124a5255c9d4964ee70  
Author: Brandon Lind <lind.brandon.m@gmail.com>  
Date:   Tue Sep 5 11:47:40 2023 -0400
#########################################################



In [2]:
outerdir = '/home/b.lind/offsets/run_20220919_nuisance'
tdir = f'{outerdir}/tutorial'

rda_dir = makedir(f'{tdir}/rda')
rda_slimdir = makedir(f'{rda_dir}/rda_slimdir')
catdir = makedir(f'{rda_dir}/rda_catfiles')
shdir = makedir(f'{rda_dir}/shfiles')

shdir

'/home/b.lind/offsets/run_20220919_nuisance/tutorial/rda/shfiles'

In [3]:
# symlink mutsfile
src_rda_slimdir = '/home/b.lind/offsets/run_20220919_tutorial/rda/rda_slimdir'
basename = 'tutorial_Rout_muts_full.txt'

src = f'{src_rda_slimdir}/{basename}'
dst = f'{rda_slimdir}/{basename}'

if not op.exists(dst):
    os.symlink(src, dst)
    
dst

'/home/b.lind/offsets/run_20220919_nuisance/tutorial/rda/rda_slimdir/tutorial_Rout_muts_full.txt'

In [4]:
outerdir

'/home/b.lind/offsets/run_20220919_nuisance'

In [7]:
# symlink PCA dir
src_pca = '/home/b.lind/offsets/run_20220919_tutorial/pca'
dst_pca = f'{tdir}/pca'

if not op.exists(dst_pca):
    os.symlink(src_pca, dst_pca)
    
dst_pca

'/home/b.lind/offsets/run_20220919_nuisance/tutorial/pca'

## continue in R

In [1]:
options(run.main=FALSE)
source('/home/b.lind/code/MVP-offsets/01_src/MVP_pooled_pca_and_rda.R')
source('/home/b.lind/code/MVP-offsets/01_src/MVP_12_RDA_offset.R')

Loading required package: permute

Loading required package: lattice


Attaching package: ‘lattice’


The following object is masked from ‘package:LEA’:

    barchart


This is vegan 2.6-2

Loading required package: fit.models



R version 4.0.3 (2020-10-10)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: CentOS Linux 7 (Core)

Matrix products: default
BLAS/LAPACK: /work/lotterhos/brandon/anaconda3/envs/MVP_env_R4.0.3/lib/libopenblasp-r0.3.17.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] progress_1.2.2    qvalue_2.22.0     robust_0.7-0      fit.models_0.64  
[5] vegan_2.6-2       lattice_0.20-45   permute_0.9-7     data.table_1.14.2
[9] LEA_3.8.0        

loaded via a namespace (and not attached):
 [1] pbdZMQ_0.3-7      repr_1.1.4        reshape2_1.4.4    spline

In [2]:
seed <<- 'tutorial'
outerdir <<-        '/home/b.lind/offsets/run_20220919_nuisance/tutorial'
slimdir <<-         '/home/b.lind/offsets/run_20220919_nuisance/tutorial/rda/rda_slimdir'
snpfile <<-         '/home/b.lind/offsets/run_20220919_nuisance/tutorial/gradient_forests/training/training_files/tutorial_Rout_Gmat_sample_maf-gt-p01_GFready_pooled_all.txt'
gf_training_dir <<- '/home/b.lind/offsets/run_20220919_nuisance/tutorial/gradient_forests/training/training_files'

mvp_dir <<- '/home/b.lind/code/MVP-offsets/01_src'
ind_or_pooled <<- 'pooled'

In [3]:
# similar to MVP_pooled_pca_and_rda.R::run_rda
    # but for 9 traits
run_rda = function(snps){
    #----------------------------------------------------------------------------------------------------#
    # Run structure-corrected and -uncorrected RDA.
    # 
    # Parameters
    # ----------
    # snps - data.frame; output from `prep_snps` - pops for rows, loci for cols, frequencies for entries
    #
    # Notes
    # -----
    # - structure correction is done from PCs estimated using all loci
    #----------------------------------------------------------------------------------------------------#

    # get current environmental data
    env_pres = get_curr_envdata(ntraits=9)
    env1_mat = env_pres[, 'env1_mat']
    env2_MTWetQ = env_pres[, 'env2_MTWetQ']
    env3_MTDQ = env_pres[, 'env3_MTDQ']
    env4_PDM = env_pres[, 'env4_PDM']
    env5_PwarmQ = env_pres[, 'env5_PwarmQ']
    env6_PWM = env_pres[, 'env6_PWM']
    ISO = env_pres[, 'ISO']
    TSsd = env_pres[, 'TSsd']
    PSsd = env_pres[, 'PSsd']
    
    # read in PCA data
    pcdata = get_pc_data(env_pres)
    
    # structure-uncorrected RDA
    rdaout <- rda(snps ~ env1_mat + env2_MTWetQ + env3_MTDQ + env4_PDM + env5_PwarmQ + env6_PWM + ISO + TSsd + PSsd)
    
    # structure-corrected RDA
    rdaout_corr = rda(
        snps ~ env1_mat + env2_MTWetQ + env3_MTDQ + env4_PDM + env5_PwarmQ + env6_PWM + ISO + TSsd + PSsd + Condition(pcdata[, 'PC1'],
                                                                                                                      pcdata[, 'PC2'])
    )
    
    # save
    rda_outdir = paste0(outerdir, '/rda/rda_files')
    if (dir.exists(rda_outdir) == FALSE){
        dir.create(rda_outdir, recursive=TRUE)
    }
    file = paste0(
        rda_outdir,
        sprintf('/%s_pooled_RDA.RDS', seed)
    )
    file_corr = paste0(
        rda_outdir,
        sprintf('/%s_pooled_RDA_structcorr.RDS', seed)
    )
    
    saveRDS(rdaout, file)
    saveRDS(rdaout_corr, file_corr)

    # create a muts file like Katie's
    create_muts_file(rdaout, rdaout_corr)
    
}

In [4]:
snps = prep_snps(snpfile)

dim(snps)

[1]   100 26371

In [5]:
gf_training_dir

[1] "/home/b.lind/offsets/run_20220919_nuisance/tutorial/gradient_forests/training/training_files"

In [6]:
run_rda(snps)



Reading in current environmental data ...

saved pooled muts file to: /home/b.lind/offsets/run_20220919_nuisance/tutorial/pca/mutfiles/tutorial_pooled_Rout_muts_full.txt

In [7]:
read.table('/home/b.lind/offsets/run_20220919_nuisance/tutorial/pca/mutfiles/tutorial_pooled_Rout_muts_full.txt')

V1,V2,V3,V4,V5,V6,V7,V8,V9
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
RDA1_score,RDA2_score,RDA1_score_corr,RDA2_score_corr,RDA_mlog10P,RDA_mlog10P_sig,RDA_mlog10P_corr,RDA_mlog10P_sig_corr,mutname
-0.00255772843617841,-0.0171763866403642,-0.0155197293134619,-0.0101556589575552,0.401881941392026,FALSE,0.809784841543327,FALSE,1-8
-0.00796354510368561,-0.00791826328206935,-0.0123081286943766,-0.0133014190497065,0.149018916340669,FALSE,0.784917453623682,FALSE,1-27
-0.013594836088499,0.0141357441640421,0.023118374796771,-0.0327611118382568,0.423137939560245,FALSE,1.57939430698224,FALSE,1-34
0.0330350887896987,0.0167472384595999,0.0192723958066766,-0.0102606821399539,2.0834597231557,FALSE,0.410661648680955,FALSE,1-81
0.0143674274509156,0.00326490098066085,0.000673604564026463,0.00376193906065503,0.378934421038534,FALSE,0.007676725602666,FALSE,1-97
0.0320685245242636,0.0122588108322945,0.0173967046002657,-0.00814758232873468,1.8169684296525,FALSE,0.31252217456768,FALSE,1-153
0.00359018459099036,-0.0147633369733344,-0.0119698583542759,0.0146714962596646,0.332345723348939,FALSE,0.359785367137522,FALSE,1-154
-0.0144131121422926,0.0149120222691038,0.0250847948004276,-0.0355177627214198,0.477655276882032,FALSE,1.85279510715731,FALSE,1-206


### estimate offset in R

use basic functionality of MVP_12_RDA_offset.R

In [1]:
options(run.main=FALSE)
source('/home/b.lind/code/MVP-offsets/01_src/MVP_12_RDA_offset.R')

Loading required package: permute

Loading required package: lattice


Attaching package: ‘lattice’


The following object is masked from ‘package:LEA’:

    barchart


This is vegan 2.6-2



R version 4.0.3 (2020-10-10)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: CentOS Linux 7 (Core)

Matrix products: default
BLAS/LAPACK: /work/lotterhos/brandon/anaconda3/envs/MVP_env_R4.0.3/lib/libopenblasp-r0.3.17.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] vegan_2.6-2       lattice_0.20-45   permute_0.9-7     LEA_3.8.0        
[5] progress_1.2.2    data.table_1.14.2

loaded via a namespace (and not attached):
 [1] pillar_1.7.0      compiler_4.0.3    prettyunits_1.1.1 base64enc_0.1-3  
 [5] tools_4.0.3       digest_0.6.29     uuid_

In [2]:
outerdir <<- '/home/b.lind/offsets/run_20220919_nuisance/tutorial'
seed <<- 'tutorial'
ind_or_pooled <<- 'pooled'
slimdir <<- paste0(outerdir, '/rda/rda_slimdir')
gf_training_dir <<- paste0(outerdir, '/gradient_forests/training/training_files')

dir.exists(slimdir)

[1] TRUE

In [3]:
dir.exists(gf_training_dir)

[1] TRUE

In [4]:
# created in previous section
rda_files = list.files(paste0(outerdir, '/rda/rda_files'), full.names=T)

rda_files

[1] "/home/b.lind/offsets/run_20220919_nuisance/tutorial/rda/rda_files/tutorial_pooled_RDA_structcorr.RDS"
[2] "/home/b.lind/offsets/run_20220919_nuisance/tutorial/rda/rda_files/tutorial_pooled_RDA.RDS"

### functions

In [5]:
# overwrite function from MVP_12_RDA_offset.R
get_garden_climates <- function(x){
    # garden_climates is same thing as env_pres (as long as pooled)
    return(data.frame(env_pres))
}

In [6]:
# overwrite function from MVP_12_RDA_offset.R
    # updated from 06_train_RDA so that ntraits==9 and equations have nuisance envs
train_outlier_rda = function(outlier_snps, rda, env_pres, ntraits=6){
    #---------------------------------------------------------------#
    # Train new RDA on outliers inferred from script input args.
    # 
    # Parameters
    # ----------
    # outlier_snps
    #     - output from subset_snps
    # rda
    #     - object loaded from rda_file input arg to this script
    # ntraits
    #     - the number of traits (environments) under selection in the simulation seed
    # 
    # Notes
    # -----
    # - assumes any structure correction is done with first two PCs
    #     - see get_pc_data()
    #---------------------------------------------------------------#
    cat(sprintf('\n\nTraining outlier RDA ...'))

    # decide if I need to estimate a new RDA
    if (use_RDA_outliers == FALSE & ntraits==9){
        # just use the original RDA object - this could be structure-corrected or not
        outlier_rda = rda

    } else {  # estimate a new RDA object with `outlier_snps`

        if (grepl('_RDA_structcorr.RDS', basename(rda_file))){  # this is structure corrected
            # get PCs
            pcdata = get_pc_data(env_pres)

            # replicate Variables object from capblancq & forester
            Variables = data.frame(pcdata, env_pres)

            # get RDA
            outlier_rda = rda(
                outlier_snps ~ env1_mat + env2_MTWetQ + env3_MTDQ + env4_PDM + env5_PwarmQ + env6_PWM + ISO + TSsd + PSsd + Condition(PC1 + PC2),
                Variables
            )


        } else {  # this is not structure corrected
            # replicate Variables object from capblancq & forester
            Variables = data.frame(env_pres)

            # get RDA
            outlier_rda = rda(
                outlier_snps ~ env1_mat + env2_MTWetQ + env3_MTDQ + env4_PDM + env5_PwarmQ + env6_PWM + ISO + TSsd + PSsd,
                Variables
            )
        }
    }

    return(outlier_rda)
}

In [7]:
# rewrite main function from MVP_12_RDA_offset.R
main <- function(use_RDA_outliers, rda_file, ntraits=9){
    print(c(use_RDA_outliers, rda_file))
    
    # read in RDA object trained on all loci (either structure-corrected or not)
#     rda_file <<- run_rda(outlier_snps)  # rda_file needed for MVP_12_RDA_offset.R functions
    rda = readRDS(rda_file)
    
    # get the SNPs, subset for outliers (which would be all snps if use_RDA_outliers==FALSE)
    outlier_snps = subset_snps()
    print(c('len(outlier_snps) = ', len(outlier_snps)))

    
    # below here is basically MVP_12_RDA_offset.R::offset_estimation
    
    # get current environmental data
    env_pres <<- get_curr_envdata(ntraits=ntraits)
    
    # train RDA on outliers
    outlier_rda = train_outlier_rda(outlier_snps, rda, env_pres, ntraits=ntraits)
    
    # estimate RDA offset for each of the gardens on the simulated landscape
        # 'None' for `subset` because `subset` is passed to estimate_garden_offsets::get_garden_climates
    estimate_garden_offsets('None', outlier_rda, env_pres, ntraits=ntraits)
    
}

## loop over loci sets

In [8]:
for (rda_file in rda_files){
    for (use_RDA_outliers in c('FALSE', 'CAUSAL', 'NEUTRAL', 'TRUE')){
        
        main(use_RDA_outliers, rda_file)
        
    }
}

[1] "FALSE"                                                                                               
[2] "/home/b.lind/offsets/run_20220919_nuisance/tutorial/rda/rda_files/tutorial_pooled_RDA_structcorr.RDS"


Reading in SNPs ...
[1] "mutsfile ="                                                                                         
[2] "/home/b.lind/offsets/run_20220919_nuisance/tutorial/pca/mutfiles/tutorial_pooled_Rout_muts_full.txt"
[1] "len(outliers) = " "26371"           
[1] "len(outlier_snps) = " "26371"               


Reading in current environmental data ...

Training outlier RDA ...


estimating offset across gardens [=>-----------------------------------------] 5/100 (  5%) eta:  4s

estimating offset across gardens [==>----------------------------------------] 6/100 (  6%) eta:  5s

estimating offset across gardens [==>----------------------------------------] 7/100 (  7%) eta:  5s

estimating offset across gardens [==>----------------------------------------] 8/100 (  8%) eta:  5s

estimating offset across gardens [===>---------------------------------------] 9/100 (  9%) eta:  5s

estimating offset across gardens [===>--------------------------------------] 10/100 ( 10%) eta:  5s

estimating offset across gardens [====>-------------------------------------] 11/100 ( 11%) eta:  5s

estimating offset across gardens [====>-------------------------------------] 12/100 ( 12%) eta:  5s

estimating offset across gardens [====>-------------------------------------] 13/100 ( 13%) eta:  5s

estimating offset across gardens [=====>------------------------------------] 14/


estimating offset across gardens [===================================>------] 86/100 ( 86%) eta:  1s

estimating offset across gardens [====================================>-----] 87/100 ( 87%) eta:  1s

estimating offset across gardens [====================================>-----] 88/100 ( 88%) eta:  1s

estimating offset across gardens [====================================>-----] 89/100 ( 89%) eta:  1s

estimating offset across gardens [=====================================>----] 90/100 ( 90%) eta:  1s

estimating offset across gardens [=====================================>----] 91/100 ( 91%) eta:  1s

estimating offset across gardens [======================================>---] 92/100 ( 92%) eta:  1s

estimating offset across gardens [======================================>---] 93/100 ( 93%) eta:  0s

estimating offset across gardens [======================================>---] 94/100 ( 94%) eta:  0s

estimating offset across gardens [=======================================>--] 95/



wrote offset to: /home/b.lind/offsets/run_20220919_nuisance/tutorial/rda/offset_outfiles/tutorial_pooled_FALSE_ntraits-9_structcorr_rda_offset.txt

wrote rda list to: /home/b.lind/offsets/run_20220919_nuisance/tutorial/rda/offset_outfiles/tutorial_pooled_FALSE_ntraits-9_structcorr_rda_list.RDS[1] "CAUSAL"                                                                                              
[2] "/home/b.lind/offsets/run_20220919_nuisance/tutorial/rda/rda_files/tutorial_pooled_RDA_structcorr.RDS"


Reading in SNPs ...
[1] "mutsfile ="                                                                                     
[2] "/home/b.lind/offsets/run_20220919_nuisance/tutorial/rda/rda_slimdir/tutorial_Rout_muts_full.txt"
[1] "len(outliers) = " "90"              
[1] "len(outlier_snps) = " "90"                  


Reading in current environmental data ...

Training outlier RDA ...


estimating offset across gardens [=>-----------------------------------------] 5/100 (  5%) eta:  5s

estimating offset across gardens [==>----------------------------------------] 6/100 (  6%) eta:  5s

estimating offset across gardens [==>----------------------------------------] 7/100 (  7%) eta:  5s

estimating offset across gardens [==>----------------------------------------] 8/100 (  8%) eta:  5s

estimating offset across gardens [===>---------------------------------------] 9/100 (  9%) eta:  5s

estimating offset across gardens [===>--------------------------------------] 10/100 ( 10%) eta:  5s

estimating offset across gardens [====>-------------------------------------] 11/100 ( 11%) eta:  5s

estimating offset across gardens [====>-------------------------------------] 12/100 ( 12%) eta:  5s

estimating offset across gardens [====>-------------------------------------] 13/100 ( 13%) eta:  5s

estimating offset across gardens [=====>------------------------------------] 14/

...


estimating offset across gardens [===================================>------] 86/100 ( 86%) eta:  1s

estimating offset across gardens [====================================>-----] 87/100 ( 87%) eta:  1s

estimating offset across gardens [====================================>-----] 88/100 ( 88%) eta:  1s

estimating offset across gardens [====================================>-----] 89/100 ( 89%) eta:  1s

estimating offset across gardens [=====================================>----] 90/100 ( 90%) eta:  1s

estimating offset across gardens [=====================================>----] 91/100 ( 91%) eta:  1s

estimating offset across gardens [======================================>---] 92/100 ( 92%) eta:  1s

estimating offset across gardens [======================================>---] 93/100 ( 93%) eta:  0s

estimating offset across gardens [======================================>---] 94/100 ( 94%) eta:  0s

estimating offset across gardens [=======================================>--] 95/



wrote offset to: /home/b.lind/offsets/run_20220919_nuisance/tutorial/rda/offset_outfiles/tutorial_pooled_NEUTRAL_ntraits-9_nocorr_rda_offset.txt

wrote rda list to: /home/b.lind/offsets/run_20220919_nuisance/tutorial/rda/offset_outfiles/tutorial_pooled_NEUTRAL_ntraits-9_nocorr_rda_list.RDS[1] "TRUE"                                                                                     
[2] "/home/b.lind/offsets/run_20220919_nuisance/tutorial/rda/rda_files/tutorial_pooled_RDA.RDS"


Reading in SNPs ...
[1] "mutsfile ="                                                                                         
[2] "/home/b.lind/offsets/run_20220919_nuisance/tutorial/pca/mutfiles/tutorial_pooled_Rout_muts_full.txt"
[1] "len(outliers) = " "2325"            
[1] "len(outlier_snps) = " "2325"                


Reading in current environmental data ...

Training outlier RDA ...


estimating offset across gardens [=>-----------------------------------------] 5/100 (  5%) eta:  5s

estimating offset across gardens [==>----------------------------------------] 6/100 (  6%) eta:  5s

estimating offset across gardens [==>----------------------------------------] 7/100 (  7%) eta:  5s

estimating offset across gardens [==>----------------------------------------] 8/100 (  8%) eta:  5s

estimating offset across gardens [===>---------------------------------------] 9/100 (  9%) eta:  5s

estimating offset across gardens [===>--------------------------------------] 10/100 ( 10%) eta:  5s

estimating offset across gardens [====>-------------------------------------] 11/100 ( 11%) eta:  5s

estimating offset across gardens [====>-------------------------------------] 12/100 ( 12%) eta:  5s

estimating offset across gardens [====>-------------------------------------] 13/100 ( 13%) eta:  5s

estimating offset across gardens [=====>------------------------------------] 14/


estimating offset across gardens [===================================>------] 86/100 ( 86%) eta:  1s

estimating offset across gardens [====================================>-----] 87/100 ( 87%) eta:  1s

estimating offset across gardens [====================================>-----] 88/100 ( 88%) eta:  1s

estimating offset across gardens [====================================>-----] 89/100 ( 89%) eta:  1s

estimating offset across gardens [=====================================>----] 90/100 ( 90%) eta:  1s

estimating offset across gardens [=====================================>----] 91/100 ( 91%) eta:  1s

estimating offset across gardens [======================================>---] 92/100 ( 92%) eta:  0s

estimating offset across gardens [======================================>---] 93/100 ( 93%) eta:  0s

estimating offset across gardens [======================================>---] 94/100 ( 94%) eta:  0s

estimating offset across gardens [=======================================>--] 95/



wrote offset to: /home/b.lind/offsets/run_20220919_nuisance/tutorial/rda/offset_outfiles/tutorial_pooled_TRUE_ntraits-9_nocorr_rda_offset.txt

wrote rda list to: /home/b.lind/offsets/run_20220919_nuisance/tutorial/rda/offset_outfiles/tutorial_pooled_TRUE_ntraits-9_nocorr_rda_list.RDS

In [10]:
read.table('/home/b.lind/offsets/run_20220919_nuisance/tutorial/rda/offset_outfiles/tutorial_pooled_TRUE_ntraits-9_nocorr_rda_offset.txt')

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,⋯,X91,X92,X93,X94,X95,X96,X97,X98,X99,X100
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.0000000,0.2062939,0.4911793,0.8960459,0.8380601,0.6662669,0.3523105,0.3460008,0.5704100,1.2359627,⋯,1.9518646,1.6138718,1.3838652,1.3178687,1.7813676,2.67000023,3.626141,4.543887,5.437206,5.696564
2,0.2062939,0.0000000,0.3548076,0.9074510,0.8671177,0.7426265,0.3160454,0.4002114,0.6376810,1.3499456,⋯,2.0553435,1.7244584,1.4947651,1.4400014,1.9096487,2.80918878,3.770554,4.689483,5.585651,5.844652
3,0.4911793,0.3548076,0.0000000,0.6756064,0.6742060,0.6912914,0.4624242,0.5645858,0.6146905,1.2878379,⋯,1.9303367,1.6448983,1.4399647,1.4072071,1.8556426,2.75104800,3.709473,4.624826,5.524992,5.782261
4,0.8960459,0.9074510,0.6756064,0.0000000,0.1538517,0.5181167,0.8994066,0.8802997,0.5811209,0.8002656,⋯,1.2994410,1.1094814,0.9829074,0.9714811,1.3190430,2.16377809,3.101894,4.008100,4.908204,5.163348
5,0.8380601,0.8671177,0.6742060,0.1538517,0.0000000,0.3729974,0.8179987,0.7786134,0.4494051,0.6949376,⋯,1.2638110,1.0234320,0.8711875,0.8523134,1.2352540,2.10880962,3.062671,3.974977,4.875051,5.130914
6,0.6662669,0.7426265,0.6912914,0.5181167,0.3729974,0.0000000,0.5994387,0.5020168,0.1335476,0.6433093,⋯,1.3620152,1.0049249,0.7780962,0.7303477,1.2047952,2.12334074,3.099985,4.020502,4.916785,5.174161
7,0.3523105,0.3160454,0.4624242,0.8994066,0.8179987,0.5994387,0.0000000,0.1749105,0.4755620,1.2249336,⋯,1.9552833,1.5797982,1.3301307,1.2843969,1.7798338,2.70038939,3.676377,4.596606,5.490862,5.748990
8,0.3460008,0.4002114,0.5645858,0.8802997,0.7786134,0.5020168,0.1749105,0.0000000,0.3791279,1.1008308,⋯,1.8484175,1.4587559,1.2047348,1.1486072,1.6448129,2.56294117,3.539021,4.459809,5.351904,5.610360
9,0.5704100,0.6376810,0.6146905,0.5811209,0.4494051,0.1335476,0.4755620,0.3791279,0.0000000,0.7553726,⋯,1.4840146,1.1139488,0.8757410,0.8311118,1.3167945,2.23870260,3.216432,4.136992,5.032916,5.290665


[top](#home)